In [ ]:
import os
import boto3
import pandas as pd
from io import BytesIO
from nexusadspy import AppnexusReport
from datetime import date
from nexusadspy import AppnexusClient
client = AppnexusClient('.appnexus_auth.json')

In [ ]:
def to_s3(bucket,filename,content):
    client = boto3.client('s3')
    k = folder+filename
    client.put_object(Bucket=bucket, Key=k, Body=content)

In [ ]:
## daily reports - everything is based on delivery so needs to be run daily.
## e.g. advertiser exists but has not delivered in *30* days - will not appear if replace?

dimensions = ["advertiser", "campaign", "creative", "deal", "insertion_order", "line_item", "seller_member", "site"]
bucket = "finecast-appnexus-lld"

## need to update dates when scheduling report
start_date_input = '2018-09-01'
end_date_input = str(date.today())

for i in range(0,len(dimensions)):
    columns = [dimensions[i]+"_id",
               dimensions[i]+"_name"]

    report_type = "network_analytics"

    report = AppnexusReport(start_date=start_date_input,
                            end_date=end_date_input,
                            report_type=report_type,
                            columns=columns)

    data = report.get(format_='pandas')
    
    filename = dimensions[i]+"_test.csv"
    folder = "feeds/metadata_tables/"+dimensions[i]+"/"

    csv_buffer = BytesIO()
    data.to_csv(csv_buffer, index = False)
    content = csv_buffer.getvalue()
    
    to_s3(bucket,filename,content)
    
    print(dimensions[i]+" done")

In [ ]:
## check report type
columns = ["advertiser_id",
           "advertiser_name"]

report_type = "network_analytics"

report = AppnexusReport(start_date='2016-11-07',
                          end_date='2016-11-08',
                          report_type=report_type,
                          columns=columns)
output_json = report.get()
check = report.get(format_='pandas')
check

In [ ]:
os.environ.get('USERNAME_NEXUSADSPY')

In [ ]:
## one time reports

read_service_dimensions = ["browser", "device-model", "media-subtype", "operating-system-extended"]
folder_name = ["browser", "device_model", "media_subtype", "operating_system_extended"]
bucket = "finecast-appnexus-lld"

for i in range(0,len(read_service_dimensions)):
    r = client.request(read_service_dimensions[i], 'GET')
    data = pd.DataFrame(r)
    data = data.filter(items=['id', 'name'])
    data.columns = [folder_name[i]+'_id', folder_name[i]+'_name']
    
    filename = read_service_dimensions[i]+"_test.csv"
    folder = "feeds/metadata_tables/"+folder_name[i]+"/"

    csv_buffer = BytesIO()
    data.to_csv(csv_buffer, index = False)
    content = csv_buffer.getvalue()
    
    to_s3(bucket,filename,content)
    
    print(read_service_dimensions[i]+"done")

In [ ]:
## testing read services
read_service_dimensions = ['carrier']
r = client.request('carrier', 'GET')
data = pd.DataFrame(r)
data = data.filter(items=['id', 'name'])
data.columns = [read_service_dimensions[0]+'_id', read_service_dimensions[0]+'_name']
data

In [ ]:
## one time carrier report

bucket = "finecast-appnexus-lld"

r = client.request('carrier', 'GET')
data = pd.DataFrame(r)
data = data.loc[data['country_code']=='AU']
data = data.filter(items=['id', 'name'])
data.columns = ['carrier_id', 'carrier_name']
    
filename = "carrier_test.csv"
folder = "feeds/metadata_tables/carrier/"

csv_buffer = BytesIO()
data.to_csv(csv_buffer, index = False)
content = csv_buffer.getvalue()
    
to_s3(bucket,filename,content)

In [ ]:
## one time city and dma report
## this one is too slow - filter API request

bucket = "finecast-appnexus-lld"

r = client.request('city', 'GET')
data = pd.DataFrame(r)
data = data.loc[data['country_code']=='AU']
data = data.filter(items=['id', 'name', 'dma_id', 'dma_name'])
data.columns = ['city_id', 'city_name', 'dma_id', 'dma_name']
    
filename = "city_dma_test.csv"
folder = "feeds/metadata_tables/city/"

csv_buffer = BytesIO()
data.to_csv(csv_buffer, index = False)
content = csv_buffer.getvalue()
    
to_s3(bucket,filename,content)

In [ ]:
## one time mobile_app instance report

bucket = "finecast-appnexus-lld"

r = client.request('mobile-app-instance', 'GET')
data = pd.DataFrame(r)
data = data.filter(items=['id', 'instance-bundle'])
data.columns = ['mobile_app_instance_id', 'mobile_app_instance_name']

filename = "mobile_app_instance_test.csv"
folder = "feeds/metadata_tables/mobile_app_instance/"

csv_buffer = BytesIO()
data.to_csv(csv_buffer, index = False)
content = csv_buffer.getvalue()
    
to_s3(bucket,filename,content)